### 第九课
第八课找了google brain的人[Jusin Johnson](https://research.google.com/pubs/JonathonShlens.html)过来讲东西，网上并找不到第八课具体资料。只能在第九课开头的时候看到，讲师在说“ Because of the guest lectures, we haven’t been able to get to some important concepts that
a lot of people seem to be confused about.”


#### Queues and Coordinators
这个就是说了一个多线程的事儿，他说的那么多，就是说了一句话：“多线程很重要，想知道咋用，你去看代码”

tf.Coordinator是用来多个线程一起停止并报告异常                                                
tf.train.QueueRunner是生成运行tensor的线程的

In [1]:
# 我又一次开始敲代码分析代码
# https://github.com/chiphuyen/tf-stanford-tutorials/blob/master/examples/09_queue_example.py
# 这一块我分析得也是云里雾里，待之后再回头看

import tensorflow as tf
import numpy as np

N_SAMPLES = 1000
NUM_THREADS = 4

data = 10 * np.random.randn(N_SAMPLES,4) + 1
# 生成一个[1000.4]的这么一个矩阵
# np.random.randn(d0,d1,,,,,dn) 参数为维数

target = np.random.randint(0,2,size=N_SAMPLES)
# 生成一个只有01的[1000]的矩阵
# numpy.random.randint(low, high=None, size=None, dtype='l')，low是生成中最小的数，high是上界


queue = tf.FIFOQueue(capacity=50,dtypes=[tf.float32,tf.int32],shapes=[[4],[]])
# 建立一个容量是50的队列

enqueue_op = queue.enqueue_many([data,target])  # 取出队列中的数据；我的理解，这里是把队列这个东西跟data做一个链接，把data放到queue里面

data_sample,label_sample = queue.dequeue() # 这个是创建了一个从queue里面取数据的工具


qr = tf.train.QueueRunner(queue,[enqueue_op] * NUM_THREADS) 
# 定义一个queueRunner，这个负责对线程进行操作

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess,coord = coord, start = True)
    # 获得thread
    
    for step in xrange(100):
        if coord.should_stop():
            # 如果所有的线程都停了，那么也应该停下来
            break
        data_batch,label_batch = sess.run([data_sample,label_sample])
        coord.request_stop()
        # 停下线程， 调用了request_stop()之后，调用should_stop()返回 True
        coord.join(enqueue_threads)
        


#### Data Readers
这里说了读取数据有三种形式：
- 通过常量读取
- 通过feed dict 读取
- TextLineReader

前两种，只能保证数据从storage读出，不能从device读出


In [2]:
# TFRedcord的类型：
tf.TextLineReader # 按行返回
# Outputs the lines of a file delimited by newlines
# E.g. text files, CSV files

tf.FixedLengthRecordReader  # 按固定长度返回
# Outputs the entire file when all files have same fixed lengths
# E.g. each MNIST file has 28 x 28 pixels, CIFAR-10 32 x 32 x 3

tf.WholeFileReader   # 返回去全部文件内容
# Outputs the entire file content. This is useful when each file contains a sample

tf.TFRecordReader   # 返回二进制数据
# Reads samples from TensorFlow's own binary format (TFRecord)

tf.ReaderBase   # 可以根据他重新写reader
# Allows you to create your own readers


tensorflow.python.ops.io_ops.ReaderBase

In [3]:
# 使用reader要先创建一个queue用来存这些文件的名字

filename_queue = tf.train.string_input_producer(["heart.csv"])
reader = tf.TextLineReader(skip_header_lines=1)  # skip_header_lines 是每个文件跳过开头的多少行
# it means you choose to skip the first line for every file in the queue


In [ ]:
# 这里使用之前用到的 FIFOQueue

filenames = ["heart.csv"]
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.TextLineReader(skip_header_lines=1) # skip the first line in the file
key, value = reader.read(filename_queue) # key是file的唯一标识scalar string 值 value是 string tensor
with tf.Session() as sess:
     coord = tf.train.Coordinator()
     threads = tf.train.start_queue_runners(coord=coord)
     print sess.run(key) # data/heart.csv:2
     print sess.run(value) # 144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
     coord.request_stop()
     coord.join(threads)

In [ ]:
# 把读出的数据，转换成可以feed用的数据

# record_defaults 存在的意义：
# 1、告诉decoder说需要的数据类型
# 2、产生默认值

record_defaults = [[1.0] for _ in range(N_FEATURES)] # define all features to be floats
# 定义一个默认的record类型 
record_defaults[4] = [''] # make the fifth feature string
record_defaults.append([1])
content = tf.decode_csv(value, record_defaults=record_defaults)

# have our content is a list of 10 elements, 8 are floats, 1 is string, and 1 is integer

In [ ]:
# 装换 类型
# convert the 5th column (present/absent) to the binary value 0 and 1
condition = tf.equal(content[4], tf.constant('Present'))
content[4] = tf.select(condition, tf.constant(1.0), tf.constant(0.0))
# pack all 9 features into a tensor
features = tf.pack(content[:N_FEATURES])
# assign the last column to label
label = content[-1]


In [ ]:
# 设置返回的batch多少
# minimum number elements in the queue after a dequeue, used to ensure that the samples are sufficiently mixed
# I think 10 times the BATCH_SIZE is sufficient
min_after_dequeue = 10 * BATCH_SIZE
# the maximum number of elements in the queue
capacity = 20 * BATCH_SIZE
# shuffle the data to generate BATCH_SIZE sample pairs
data_batch, label_batch = tf.train.shuffle_batch([features, label], batch_size=BATCH_SIZE,
 capacity=capacity, min_after_dequeue=min_after_dequeue)


#### TFRecord
他说二进制的文件，快，然而，我并不知道是为什么

In [ ]:
# 以下是把图片转换成byte string

def get_image_binary(filename):
     image = Image.open(filename)
     image = np.asarray(image, np.uint8)
     shape = np.array(image.shape, np.int32)
     return shape.tobytes(), image.tobytes() # convert image to raw data bytes in the array.


In [ ]:
# 把byte string 转成 TFRecord 文件

def write_to_tfrecord(label, shape, binary_image, tfrecord_file):
     """ This example is to write a sample to TFRecord file. If you want to write
     more samples, just use a loop.
     """
     writer = tf.python_io.TFRecordWriter(tfrecord_file)
     # write label, shape, and image content to the TFRecord file
     example = tf.train.Example(features=tf.train.Features(feature={
             'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[label])),
             'shape': tf.train.Feature(bytes_list=tf.train.BytesList(value=[shape])),
             'image':tf.train.Feature(bytes_list=tf.train.BytesList(
             value=[binary_image]))
         }))
     writer.write(example.SerializeToString())
     writer.close()


In [ ]:
# 读取TFRecord文件

def read_from_tfrecord(filenames):
     tfrecord_file_queue = tf.train.string_input_producer(filenames, name='queue')
     reader = tf.TFRecordReader()
     _, tfrecord_serialized = reader.read(tfrecord_file_queue)
     # label and image are stored as bytes but could be stored as
     # int64 or float64 values in a serialized tf.Example protobuf.
     tfrecord_features = tf.parse_single_example(tfrecord_serialized,
                         features={
                             'label': tf.FixedLenFeature([], tf.string),
                             'shape': tf.FixedLenFeature([], tf.string),
                             'image': tf.FixedLenFeature([], tf.string),
                         }, name='features')
     # image was saved as uint8, so we have to decode as uint8.
     image = tf. decode_raw(tfrecord_features['image'], tf.uint8)
     shape = tf.decode_raw(tfrecord_features['shape'], tf.int32)
     # the image tensor is flattened out, so we have to reconstruct the shape
     image = tf.reshape(image, shape)
     label = tf.cast(tfrecord_features['label'], tf.string)
     return label, shape, image
